<a href="https://colab.research.google.com/github/qazpoiu79/qazpoiu/blob/main/nv_shppng_pre_nrmlzd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def scrapNparse(URL):
    import requests
    from bs4 import BeautifulSoup
    
    got = requests.get(URL)
    parsed = BeautifulSoup(got.content, 'html.parser')
    
    return parsed

In [ ]:
categories = scrapNparse('https://search.shopping.naver.com/best100v2/main.nhn').select('div#content > div.h_area > ul#snb > li')

category_info = dict({
    'titles' : list(),
    'links' : list(),
    'ids' : list()
})


for category in categories:
    category_title = category.select_one('a > img')['alt'].replace('/','|')
    category_info['titles'].append(category_title)
    print(category_title)
    
    category_link = 'https://search.shopping.naver.com'+category.select_one('a')['href']
    category_info['links'].append(category_link)
    print(category_link)
    
    category_id = category['catid']
    category_info['ids'].append(category_id)
    print(category_id)

패션의류
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000000
50000000
패션잡화
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000001
50000001
화장품|미용
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000002
50000002
디지털|가전
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000003
50000003
가구|인테리어
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000004
50000004
식품
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000006
50000006
스포츠|레저
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000007
50000007
출산|육아
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000005
50000005
생활|건강
https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000008
50000008


In [ ]:
categories = list()
for title, link, ide in zip(category_info['titles'],category_info['links'],category_info['ids']):
    print(title, link, ide)
    categories.append((title, link, ide))

패션의류 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000000 50000000
패션잡화 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000001 50000001
화장품|미용 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000002 50000002
디지털|가전 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000003 50000003
가구|인테리어 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000004 50000004
식품 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000006 50000006
스포츠|레저 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000007 50000007
출산|육아 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000005 50000005
생활|건강 https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000008 50000008


In [ ]:
####################################################################################################
# BEST100을 전부 다 긁어오는 것은 시간도 자원도 너무 많이 소모되어 비용-효익 대응이 맞지 않다.     #
# 학습목적이므로 패션의류 > 여성의류, 남성의류만 수집한다.                                         #
####################################################################################################

## 스크랩한 날짜별로 디렉토리를 만들고 그 안에 스트랩한 CSV파일을 저장함. ##
import os
scrap_date = scrapNparse('https://search.shopping.naver.com/best100v2/main.nhn').select_one('div#wrap > div#container > div#content > div#base_area > div.curr_date > p.ymd').get_text().rstrip('.').replace('.','-')

if scrap_date not in os.listdir('/content/drive/MyDrive/Colab Notebooks/nv_shppng/CSVs/'):
  os.mkdir('/content/drive/MyDrive/Colab Notebooks/nv_shppng/CSVs/'+scrap_date)
else:
  pass

import csv

whole_tbl = list()

for a, (category_title, category_link) in enumerate(zip(category_info['titles'], category_info['links']), start=1):
  # 카테고리 진입
  print(category_title)
  # for b, subcategory in enumerate(scrapNparse(category_link).select('div.category_lst > div.srh_category > ul > li > a')[0:1], start=1):  # 인덱스 [0:1] : 여성의류
  for b, subcategory in enumerate(scrapNparse(category_link).select('div.category_lst > div.srh_category > ul > li > a')[2:3], start=1):  # 인덱스 [2:3] : 남성의류
    # 카테고리 > 하위카테고리 진입
    subcategory_title = subcategory.select_one('em').get_text().replace('\t','').replace('\n','').replace('/',',')
    subcategory_link = 'https://search.shopping.naver.com/'+subcategory['href']
    print(subcategory_title)

    for c, subsubcategory in enumerate(scrapNparse(subcategory_link).select('div.category_lst > div.srh_category > ul > li > a'), start=1):
      # 카테고리 > 하위카테고리 > 세부카테고리 진입
      category_id = int(str(a).ljust(2, '0')+str(b).zfill(2)+str(c).zfill(2))
      subsubcategory_title = subsubcategory.select_one('em').get_text().replace('\t','').replace('\n','').replace('/',',')
      subsubcategory_link = 'https://search.shopping.naver.com/'+subsubcategory['href']
      subsubcategory_tbl = list()
      print(subsubcategory_title)

      try:
        for rank, item in enumerate(scrapNparse(subsubcategory_link).select('div#wrap_productListArea > div#productListArea > ul > li'), start=1):
          # 카테고리 > 하위카테고리 > 세부카테고리 > 개별 상품 썸네일 선택
          item_mid = item['data-nv-mid']
          item_title = item.select_one('p > a')['title']
          item_price = int(item.select_one('div.price > strong > span.num').get_text().replace(',', ''))
          item_brand, item_regdate = '', ''
          item_link = item.select_one('p > a')['href']
          print(item_mid, item_title, item_price)
          
          for item_info in scrapNparse(item_link).select('div.style_inner__1Eo2z > div.top_summary_title__15yAr > div.top_info_inner__1cEYE > span.top_cell__3DnEV'):
            # 카테고리 > 하위카테고리 > 세부카테고리 > 개별 상품 페이지 진입
            if item_info.get_text().count('브랜드') != 0 and item_info.get_text().count('브랜드 카탈로그') != 1:
              item_brand = item_info.select_one('em').get_text()
            elif item_info.get_text().count('등록일') != 0:
              item_regdate = item_info.select_one('em').get_text().rstrip('.').replace('.', '-')
            else:
              pass
            
          print((category_title, subcategory_title, subsubcategory_title, rank, item_mid, item_title, item_price, item_brand, item_regdate))
          subsubcategory_tbl.append((category_title, subcategory_title, subsubcategory_title, rank, item_mid, item_title, item_price, item_brand, item_regdate))
          print(subsubcategory_tbl[len(subsubcategory_tbl)-1])


      except:
        print('Error; Probably the item is prohibited for under-ages.')
        pass
    


      # 세부카테고리를 CSV파일로 저장
      file_name = '/content/drive/MyDrive/Colab Notebooks/nv_shppng/CSVs/'+scrap_date+'/'+str(category_title)+'|'+str(subcategory_title)+'|'+str(subsubcategory_title)+'.csv'
      with open(file_name, 'w', encoding='utf-8-sig', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for record in subsubcategory_tbl:
          writer.writerow(record)




스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
('패션의류', '남성의류', '코트', 53, '26212415682', '애드호크 가을 2컬러 세미오버 싱글 트렌치코트 W2BB3044', 25650, '애드호크', '2021-03')
26423293239 휠라 코트디럭스 1TM00651D 151 P301874685 41590
('패션의류', '남성의류', '코트', 54, '26423293239', '휠라 코트디럭스 1TM00651D 151 P301874685', 41590, '휠라', '2021-03')
('패션의류', '남성의류', '코트', 54, '26423293239', '휠라 코트디럭스 1TM00651D 151 P301874685', 41590, '휠라', '2021-03')
23605729093 닉스 남성오버핏 맥코트 NOSCCV101CH 31200
('패션의류', '남성의류', '코트', 55, '23605729093', '닉스 남성오버핏 맥코트 NOSCCV101CH', 31200, '닉스', '2020-07')
('패션의류', '남성의류', '코트', 55, '23605729093', '닉스 남성오버핏 맥코트 NOSCCV101CH', 31200, '닉스', '2020-07')
27348079119 앤드지 구스다운 하이넥 코트 BLA4CP1501 59000
('패션의류', '남성의류', '코트', 56, '27348079119', '앤드지 구스다운 하이넥 코트 BLA4CP1501', 59000, '앤드지', '2021-05')
('패션의류', '남성의류', '코트', 56, '27348079119', '앤드지 구스다운 하이넥 코트 BLA4CP1501', 59000, '앤드지', '2021-05')
25899672708 오플 키아X원빈 트렌치코트 남성 21SPRING 58650
('패션의류', '남성의류', '코트', 57, '25899672708', '오플 키아X원빈 트렌치코트 남성 21SPRING', 58650, '', '2